### Fact Table


In [0]:
df=spark.sql("select * from catabricks_cat.silver.orders_silver")

In [0]:
df_dimcus=spark.sql("select DimCustomerKey,customer_id as dim_customer_id from catabricks_cat.gold.dimcustomers")
df_dimprod=spark.sql("select product_id as DimProductKey,product_id as dim_customer_id from catabricks_cat.gold.dimproduct")

### Fact Dataframe

In [0]:
df_fact=df.join(dimcus,df['customer_id']==df_dimcus['dim_customer_id']).join(dimprod,df['product_id']==df_dimprod['dim_product_id']).select(df['*'],df_dimcus['DimCustomerKey'],how='left')
df_fact_new=df_fact.drop("dim_customer_id","dim_product_id","customer_id","product_id")

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("catabricks_cat.gold.FactOrders"):
  dlt_obj= DeltaTable.forName(spark,"catabricks_cat.gold.FactOrders")
  dlt_obj.alias("trg").merge(df_fact_new.alias("src"),"trg.order_id=src.order_id" and trg.DimCustomerKey=src.DimCustomerKey and trg.DimProductKey=src.DimProductKey)\
      whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else :
    df_fact_new.write.format("delta").option("path","abfss://gold@catabricksdatalake.dfs.core.windows.net/FactOrders").saveAsTable("catabricks_cat.gold.FactOrders")